In [ ]:
# Mutations
import pandas as pd
import requests
import json
import re, time
import math

In [ ]:
url = 'https://api.gdc.cancer.gov/v0/graphql'

headers = {"Content-Type": "application/json"}

In [ ]:
Project_ID = 'TCGA-BRCA'

### 1. Go to the Exploration Page and download the JSON from the Genes Table using Filters

In [ ]:
# Load in the json that was downloaded from the portal
with open("genes.2019-11-07.json", "r") as read_file:
    data = json.load(read_file)

In [ ]:
# Loop through all the genes in the json
Genes = []
Gene_Ids = []
for gene in data:
    Gene_Ids.append(gene['gene_id'])
    Genes.append([gene['gene_id'],
                  gene['symbol'],
                  gene['name']
                ])

###  2. Get the CNV and SSM Totals using Graphql

In [ ]:
query = '''query GenesTable_relayQuery(
  $genesTable_filters: FiltersArgument
  $genesTable_size: Int
  $genesTable_offset: Int
  $score: String
  $ssmCase: FiltersArgument
  $geneCaseFilter: FiltersArgument
  $ssmTested: FiltersArgument
  $cnvTested: FiltersArgument
  $cnvGainFilters: FiltersArgument
  $cnvLossFilters: FiltersArgument
) {
  genesTableViewer: viewer {
    explore {
      cases {
        hits(first: 0, filters: $ssmTested) {
          total
        }
      }
      filteredCases: cases {
        hits(first: 0, filters: $geneCaseFilter) {
          total
        }
      }
      cnvCases: cases {
        hits(first: 0, filters: $cnvTested) {
          total
        }
      }
      genes {
        hits(first: $genesTable_size, offset: $genesTable_offset, filters: $genesTable_filters, score: $score) {
          total
          edges {
            node {
              gene_id
              id
              symbol
              name
              cytoband
              biotype
              numCases: score
              is_cancer_gene_census
              ssm_case: case {
                hits(first: 0, filters: $ssmCase) {
                  total
                }
              }
              cnv_case: case {
                hits(first: 0, filters: $cnvTested) {
                  total
                }
              }
              case_cnv_gain: case {
                hits(first: 0, filters: $cnvGainFilters) {
                  total
                }
              }
              case_cnv_loss: case {
                hits(first: 0, filters: $cnvLossFilters) {
                  total
                }
              }
            }
          }
        }
      }
    }
  }
}'''

In [ ]:
variables = '''{
    "genesTable_filters": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            }
        ]
    },
    "genesTable_size": 10,
    "genesTable_offset": 0,
    "score": "case.project.project_id",
    "ssmCase": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "ssm"
                    ]
                }
            },
            {
                "op": "NOT",
                "content": {
                    "field": "genes.case.ssm.observation.observation_id",
                    "value": "MISSING"
                }
            }
        ]
    },
    "geneCaseFilter": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "ssm"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            }
        ]
    },
    "ssmTested": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "ssm"
                    ]
                }
            }
        ]
    },
    "cnvTested": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "cnv"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            }
        ]
    },
    "cnvGainFilters": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "cnv"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cnvs.cnv_change",
                    "value": [
                        "Gain"
                    ]
                }
            }
        ]
    },
    "cnvLossFilters": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "cnv"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cnvs.cnv_change",
                    "value": [
                        "Loss"
                    ]
                }
            }
        ]
    }
}'''

In [ ]:
request = requests.post(url, json={'query': query, 'variables': variables}, headers=headers)
Data = request.json()
SSM_Cohort = Data['data']['genesTableViewer']['explore']['filteredCases']['hits']['total']
CNV_Total = Data['data']['genesTableViewer']['explore']['cnvCases']['hits']['total']

### 3. Get the SSMs and CNV counts for each gene (Looping throuh each gene)

In [ ]:
query = '''query GenesTable_relayQuery(
  $genesTable_filters: FiltersArgument
  $genesTable_size: Int
  $genesTable_offset: Int
  $score: String
  $ssmCase: FiltersArgument
  $geneCaseFilter: FiltersArgument
  $ssmTested: FiltersArgument
  $cnvTested: FiltersArgument
  $cnvGainFilters: FiltersArgument
  $cnvLossFilters: FiltersArgument
) {
  genesTableViewer: viewer {
    explore {
      cases {
        hits(first: 0, filters: $ssmTested) {
          total
        }
      }
      filteredCases: cases {
        hits(first: 0, filters: $geneCaseFilter) {
          total
        }
      }
      cnvCases: cases {
        hits(first: 0, filters: $cnvTested) {
          total
        }
      }
      genes {
        hits(first: $genesTable_size, offset: $genesTable_offset, filters: $genesTable_filters, score: $score) {
          total
          edges {
            node {
              gene_id
              id
              symbol
              name
              cytoband
              biotype
              numCases: score
              is_cancer_gene_census
              ssm_case: case {
                hits(first: 0, filters: $ssmCase) {
                  total
                }
              }
              cnv_case: case {
                hits(first: 0, filters: $cnvTested) {
                  total
                }
              }
              case_cnv_gain: case {
                hits(first: 0, filters: $cnvGainFilters) {
                  total
                }
              }
              case_cnv_loss: case {
                hits(first: 0, filters: $cnvLossFilters) {
                  total
                }
              }
            }
          }
        }
      }
    }
  }
}'''

In [ ]:
Mutation_List = []
for i in range(math.ceil(len(Gene_Ids)/100)):
    print('Fetching affected case counts for genes ' + str(i*100) + '-' + str(i*100+100) + '.....')
    variables = '''{
    "genesTable_filters": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "genes.gene_id",
                    "value": '''+ json.dumps(Gene_Ids[i*100:i*100+100])+'''
                }
            }
        ]
    },
    "genesTable_size": 100,
    "genesTable_offset": 0,
    "score": "case.project.project_id",
    "ssmCase": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "ssm"
                    ]
                }
            },
            {
                "op": "NOT",
                "content": {
                    "field": "genes.case.ssm.observation.observation_id",
                    "value": "MISSING"
                }
            }
        ]
    },
    "geneCaseFilter": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "ssm"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "genes.gene_id",
                    "value": '''+ json.dumps(Gene_Ids[i*100:i*100+100])+'''
                }
            }
        ]
    },
    "ssmTested": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "ssm"
                    ]
                }
            }
        ]
    },
    "cnvTested": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "cnv"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "genes.gene_id",
                    "value": '''+ json.dumps(Gene_Ids[i*100:i*100+100])+'''
                }
            }
        ]
    },
    "cnvGainFilters": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "cnv"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cnvs.cnv_change",
                    "value": [
                        "Gain"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "genes.gene_id",
                    "value": '''+ json.dumps(Gene_Ids[i*100:i*100+100])+'''
                }
            }
        ]
    },
    "cnvLossFilters": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.available_variation_data",
                    "value": [
                        "cnv"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "cnvs.cnv_change",
                    "value": [
                        "Loss"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "genes.gene_id",
                    "value": '''+ json.dumps(Gene_Ids[i*100:i*100+100])+'''
                }
            }
        ]
    }
}'''
    request = requests.post(url, json={'query': query, 'variables': variables}, headers=headers)
    Data = request.json()
    Genes_List = Data['data']['genesTableViewer']['explore']['genes']['hits']['edges']
    for gene in Genes_List:
        Gene_ID = gene['node']['gene_id']
        SSM_GDC = Data['data']['genesTableViewer']['explore']['cases']['hits']['total']
        case_cnv_gain = gene['node']['case_cnv_gain']['hits']['total']
        case_cnv_loss = gene['node']['case_cnv_loss']['hits']['total']
        case_ssm_gdc_num = gene['node']['ssm_case']['hits']['total']
        case_ssm_cohort_num = gene['node']['numCases']
        Mutation_List.append([Gene_ID,
                              case_ssm_cohort_num,
                              SSM_Cohort,
                              case_ssm_gdc_num,
                              SSM_GDC,
                              case_cnv_gain,
                              CNV_Total,
                              case_cnv_loss,
                              CNV_Total])

In [ ]:
Graphql_Data = pd.DataFrame(Mutation_List,columns=['gene_id','num cases cohort','total cases cohort','num cases gdc','total cases gdc','cnv gain cases','cnv gain total','cnv loss cases','cnv loss total'])
Graphql_Data.head()

In [ ]:
Basic_Data = pd.DataFrame(Genes,columns=['gene_id','symbol','name'])
Basic_Data.head()

In [ ]:
Final_DataFrame = pd.merge(Basic_Data, Graphql_Data, on=["gene_id"],how='outer')
Final_DataFrame.head()

### 4. Get the Total Mutation Counts

In [ ]:
query = '''query SsmsAggregations_relayQuery(
  $ssmCountsfilters: FiltersArgument
) {
  ssmsAggregationsViewer: viewer {
    explore {
      ssms {
        aggregations(filters: $ssmCountsfilters, aggregations_filter_themselves: true) {
          consequence__transcript__gene__gene_id {
            buckets {
              key
              doc_count
            }
          }
        }
      }
    }
  }
}'''

In [ ]:
Mutation_Numbers = []
for i in range(math.ceil(len(Gene_Ids)/100)):
    print('Fetching affected case counts for genes ' + str(i*100) + '-' + str(i*100+100) + '.....')
    variables = '''{
    "ssmCountsfilters": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": [
                        "'''+ Project_ID +'''"
                    ]
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "consequence.transcript.gene.gene_id",
                    "value": '''+ json.dumps(Gene_Ids[i*100:i*100+100])+'''
                }
            },
            {
                "op": "in",
                "content": {
                    "field": "genes.gene_id",
                    "value": '''+ json.dumps(Gene_Ids[i*100:i*100+100])+'''
                }
            }
        ]
    }
}'''
    request = requests.post(url, json={'query': query, 'variables': variables}, headers=headers)
    Data = request.json()
    Mutation_Counts = Data['data']['ssmsAggregationsViewer']['explore']['ssms']['aggregations']['consequence__transcript__gene__gene_id']['buckets']
    for count in Mutation_Counts:
        Mutation_Numbers.append([count['key'],
                                count['doc_count']])

In [ ]:
Mutations_DF = pd.DataFrame(Mutation_Numbers,columns=['gene_id','Num Mutations'])
Mutations_DF.head()

In [ ]:
DataFrame = pd.merge(Final_DataFrame, Mutations_DF, on=["gene_id"],how='outer')
DataFrame

In [ ]:
DataFrame.sort_values(by=['num cases cohort'],ascending=False,inplace=True)

In [ ]:
DataFrame.to_csv('TCGA-BRCA_Genes_Table.tsv',sep='\t',index=False)